## Data acquisition - NIH funding data 

## Step 1 : download files 

manually downloaded from 

https://reporter.nih.gov/exporter



What does bulk RePORTER data include for ExPORTER files?
Bulk RePORTER includes data:

    Projects
    Project Abstracts – separate due to file size considerations
    Publications citing support from projects
    Link Tables for Project to Publication Associations – used to establish the many-to-many relationships between projects and publications citing support from these projects
    Patents citing support from projects
    Clinical Studies citing support from projects

Bulk ExPORTER data can be downloaded from RePORTER using the following URL format, where <fy> indicates the four-digit fiscal year:

    Projects files, (/exporter/projects/download/<fy>)
    Abstracts files, (/exporter/abstracts/download/<fy>)
    Publications files, (/exporter/publications/download/<fy>)
    Link tables files, and (/exporter/linktables/download/<fy>)
    Clinical studies files (/exporter/clinicalstudies/download)
    Patents files (/exporter/patents/download)

For example, https://reporter.nih.gov/exporter/projects/download/2020 will download Projects for FY2020.




## Step 2 : upload to Database 

In [20]:
import requests
import datetime
import os 
import time 
import pandas as pd 
import zipfile



In [ ]:
## data paths 

xFld = '/media/mike/DATA4T/_data_repository/__others/__funding_data/4000_NIH/'


xPrefix_project = 'RePORTER_PRJ_C_FY'
xPrefix_abstract = 'RePORTER_PRJABS_C_FY'


In [ ]:
## Resulting database - we use sqlite for easy replication 

xPath_DB = '/home/mike/xTemp_data_infrastructure/_staging_funding_dataset/'
xDB = xPath_DB  + 'grants_datasets.db'
xDBCon = 'sqlite:///' + xDB

In [ ]:

xPrefix_project = 'RePORTER_PRJ_C_FY'
xPrefix_abstract = 'RePORTER_PRJABS_C_FY'

# IMPORT THE TABLES -- projects 
print ('--- starting', time.ctime())
for x_year in range(2019, 2023): ## broke at 2016 
#for x_year in range(2007, 2008):
    x_year_str = str(x_year)
    print ('-------------------- process: ', x_year_str)
    xFile_Zipped = xFld + xPrefix_project + x_year_str + '.zip'
    xFile_nonZipped = xPrefix_project  + x_year_str + '.csv'
    
    if x_year in [2016, 2017, 2018]:
        xFile_nonZipped = xPrefix_project  + x_year_str + '_new.csv'
        

    xTabName = 'nih_' + x_year_str + '_projects' 
    
    #Process File 
    df1 = None
    with zipfile.ZipFile(xFile_Zipped ) as z:
        with z.open(xFile_nonZipped) as f:
            df1 = pd.read_csv(f, low_memory=False, encoding='latin-1')#, nrows=100)
            #print len(df1)
            
            df1.to_sql(name = xTabName, 
                       con = xDBCon, 
                       if_exists='replace', 
                       index=True, index_label='record_id', 
                       chunksize=10000)
    
            print ('saved ! records:', len(df1) ,  time.ctime())
    
print ('--------all done---' , time.ctime()  )



In [ ]:
## redo 2017


## data paths 
#xTabName = 'nih_2017_projects'
#xFile_Zipped = '/media/mike/DATA4T/_data_repository/__others/__funding_data/4000_NIH/RePORTER_PRJ_C_FY2017.zip'
#xFile_nonZipped= 'RePORTER_PRJ_C_FY2017_new.csv'

xTabName = 'nih_2018_projects'
xFile_Zipped = '/media/mike/DATA4T/_data_repository/__others/__funding_data/4000_NIH/RePORTER_PRJ_C_FY2018.zip'
xFile_nonZipped= 'RePORTER_PRJ_C_FY2018_new.csv'



with zipfile.ZipFile(xFile_Zipped ) as z:
    with z.open(xFile_nonZipped) as f:
        xdf1 = pd.read_csv(f, low_memory=False, encoding='latin-1', error_bad_lines=False)#, nrows=100)
        #xlines = f.readlines() 
        
xdf1.to_sql(name = xTabName, 
            con = xDBCon, 
            if_exists='replace', 
            index=True, index_label='record_id', 
            chunksize=10000)

print('saved:', len(xdf1))

In [ ]:

# abstracts 

# IMPORT THE TABLES --abstracts 
print ('--- starting', time.ctime())
for x_year in range(2019, 2023):
#for x_year in range(2007, 2008):
    x_year_str = str(x_year)
    print ('-------------------- process: ', x_year_str)
    xFile_Zipped = xFld + xPrefix_abstract  + x_year_str + '.zip'
    xFile_nonZipped = xPrefix_abstract  + x_year_str + '.csv'   
    if x_year in [2016, 2017, 2018, 2019, 2020]:
        xFile_nonZipped = xPrefix_abstract  + x_year_str + '_new.csv'    
    
    
    xTabName = 'nih_' + x_year_str + '_projects_abst' 
    
    #Process File 
    df1 = None
    with zipfile.ZipFile(xFile_Zipped ) as z:
        with z.open(xFile_nonZipped) as f:
            df1 = pd.read_csv(f, low_memory=False, encoding='latin-1')#, nrows=100)
            #print len(df1)
            
            df1.to_sql(name = xTabName, 
                       con = xDBCon, 
                       if_exists='replace', 
                       index=True, index_label='record_id', 
                       chunksize=10000)
    
            print ('saved ! records:', len(df1) ,  time.ctime())
    
print ('--------all done---' , time.ctime()  )

In [ ]:
## PARSE THE PERSON 

def process_data(df):
    new_rows = []

    for index, row in df.iterrows():
        if row.get('applicants_names'):
            application_id = row['application_id']
            applicants_names = row['applicants_names'].split(';')
            if row.get('applicants_ids'):
                applicants_ids = row['applicants_ids'].split(';')
            else:
                applicants_ids = [' ']*len(applicants_names)
                


            for app_id, app_name in zip(applicants_ids, applicants_names):
                if len(app_name.strip()) > 2:
                    xdict_data = {}
                    xdict_data['application_id'] = application_id
                    if '(contact)' in app_id:
                        xdict_data['applicant_role'] = 'contact'
                    xdict_data['applicant_id'] = app_id.replace('(contact)', '')
                    xapp_names = app_name.replace('(contact)', '')
                    if len(xapp_names.split(',')) > 1:
                        xdict_data['applicant_name_last'] = xapp_names.split(',')[0]                        
                        xdict_data['applicant_name_first'] = xapp_names.split(',')[1]
                    else:
                        xdict_data['applicant_name_last'] =  xapp_names                                 
                    xdict_data['application_id'] = application_id                           


                    new_rows.append(xdict_data)
            
    return pd.DataFrame(new_rows)



xSQL_Tpl = '''SELECT 
          APPLICATION_ID as application_id
          ,PI_IDS as applicants_ids
          ,PI_NAMEs as applicants_names
          FROM 
          nih_{}_projects'''

for xyear in [str(x) for x in list(range(2007, 2023))]:
    xSQL = xSQL_Tpl.format(xyear)

    
    xTable_name = 'nih_{}_project_investigators'.format(xyear)
    
    #print(xTable_name)
    
    xdf = pd.read_sql(xSQL, xDBCon)
    
    xdf_norm = process_data(xdf)

    xdf_norm.to_sql(name = xTable_name, 
                    con = xDBCon, 
                    if_exists='replace', 
                    index=False, 
                    chunksize=10000)
    print('saved',  xTable_name, len(xdf_norm))
    
print('done')    
    

### RESULTS 

In [13]:
## publications 

entities = ['linktables' , 'patents' , 'clinicalstudies', 'publications' , 'projects', 'abstracts']

#xentity = 'publications'
xentity = 'linktables'


xUrl_tpl = 'https://reporter.nih.gov/exporter/{}/download/'.format(xentity)

for xyear in [str(x) for x in list(range(1980, 2023))]:
    xUrl = xUrl_tpl + xyear
    #print('wget', xUrl)
    





In [ ]:
Projects files, (/exporter/projects/download/<fy>)
Abstracts files, (/exporter/abstracts/download/<fy>)
Publications files, (/exporter/publications/download/<fy>)
Link tables files, and (/exporter/linktables/download/<fy>)
Clinical studies files (/exporter/clinicalstudies/download)
Patents files (/exporter/patents/download)

For example, https://reporter.nih.gov/exporter/projects/download/2020 will download Projects for FY2020.

# Parse publications 


we only consider the linking project_pmid 




In [26]:
## Resulting database - we use sqlite for easy replication 

xPath_DB = '/home/mike/xTemp_data_infrastructure/_staging_funding_dataset/'
xDB = xPath_DB  + 'grants_datasets.db'
xDBCon = 'sqlite:///' + xDB

In [34]:
xFld = '/media/mike/DATA4T/_data_repository/__others/__funding_data/4000_NIH/__results/publication_links/'

## patterns differs REPORTER_PUBLNK_C_, RePORTER_PUBLNK_C_


#for xYear in [str(x) for x in list(range(1980, 2022))]:
for xYear in [str(x) for x in list(range(2021, 2022))]:    
    
    xFile_Zipped = xFld + xYear + '.1'
    xFile_nonZipped = 'RePORTER_PUBLNK_C_' + xYear + '.csv'
    xtab_name = 'nih_results_projpubs_link_' + xYear
    try:
        with zipfile.ZipFile(xFile_Zipped ) as z:
            with z.open(xFile_nonZipped) as f:
                df1 = pd.read_csv(f, low_memory=False, encoding='latin-1')#, nrows=100)
                df1.to_sql(xtab_name, xDBCon, index=False)
                time.sleep(1)
    
    except:
        print('ERROR !:', xYear, xFile_nonZipped)
    



ERROR !: 2021 RePORTER_PUBLNK_C_2021.csv


In [39]:
#xYear = '2021'
#xFile_nonZipped = 'RePORTER_PUBLNK_C_' + xYear + '.csv'

xYear = '2022'
xFile_nonZipped = 'RePORTER_PUBLNK_C_FY2022.csv'
xFile_Zipped = xFld + xYear + '.1'

xtab_name = 'nih_results_projpubs_link_' + xYear
with zipfile.ZipFile(xFile_Zipped ) as z:
    with z.open(xFile_nonZipped) as f:
        df1 = pd.read_csv(f, low_memory=False, encoding='latin-1')#, nrows=100)
        df1.to_sql(xtab_name, xDBCon, index=False)
        time.sleep(1)
    


In [36]:
df1.head()

,PMID,PROJECT_NUMBER
0,32491208,T32DK060414
1,31971849,P50HD093079
2,33274773,L30MH101750
3,32853834,K08EY027464
4,32394479,R01CA239251
